In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Type_1_Diabetes_Risk/GSE51058'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Risk of T1D progression in islet autoantibody positive children can be further stratified using expression patterns of multiple genes implicated in peripheral blood lymphocyte activation and function"
!Series_summary	"To identify genes with expression levels that are associated with T1D progression from AbP (islet autoantibody positive),   global gene expression changes were analyzed in AbP subjects with different T1D progression rate."
!Series_overall_design	"Total RNA were obtained from peripheral blood mononuclear cells (PBMC) of total 36 AbP subjects with different T1D progression rate.   Microarray was carried out to analyze gene expression and Kaplan-Meier survival analysis and log-rank test were used to compare differences in diabetes-free survival between groups classified based on single gene expression."
Sample Characteristics Dictionary:
{0: ['cell type: Peripheral Blood Mononuclear Cells (PBMC)'], 1: ['progressed to type 1 diabetes (t1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the background information, the dataset involves gene expression analysis

# Identify rows for clinical data
trait_row = 1  # 'progressed to type 1 diabetes (t1d) by cut-off date'
# Check unique values for age
# In the provided sample characteristics dictionary, age data is not directly available
age_row = None  
# Check unique values for gender
# In the provided sample characteristics dictionary, gender data is not directly available
gender_row = None  

# Define conversion functions
def convert_trait(value):
    try:
        after_colon = value.split(": ")[1]
        if after_colon.lower() == 'yes':
            return 1
        elif after_colon.lower() == 'no':
            return 0
    except (IndexError, ValueError):
        pass
    return None

def convert_age(value):
    return None  # Age data not available

def convert_gender(value):
    return None  # Gender data not available

# Save cohort information
save_cohort_info('GSE51058', './preprocessed/Type_1_Diabetes_Risk/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Type_1_Diabetes_Risk', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Type_1_Diabetes_Risk/trait_data/GSE51058.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1236646': [0], 'GSM1236647': [0], 'GSM1236648': [0], 'GSM1236649': [1], 'GSM1236650': [1], 'GSM1236651': [1], 'GSM1236652': [0], 'GSM1236653': [1], 'GSM1236654': [1], 'GSM1236655': [1], 'GSM1236656': [1], 'GSM1236657': [1], 'GSM1236658': [0], 'GSM1236659': [1], 'GSM1236660': [1], 'GSM1236661': [1], 'GSM1236662': [0], 'GSM1236663': [1], 'GSM1236664': [1], 'GSM1236665': [0], 'GSM1236666': [1], 'GSM1236667': [1], 'GSM1236668': [0], 'GSM1236669': [0], 'GSM1236670': [0], 'GSM1236671': [1], 'GSM1236672': [1], 'GSM1236673': [0], 'GSM1236674': [1], 'GSM1236675': [0], 'GSM1236676': [1], 'GSM1236677': [0], 'GSM1236678': [1], 'GSM1236679': [1], 'GSM1236680': [0], 'GSM1236681': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys store the identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print a preview of the gene_data
print("Gene data preview after mapping:")
print(preview_df(gene_data))


Gene data preview after mapping:
{'GSM1236646': [6.735794192, 6.901819354, 6.710040512666667, 6.976768049, 6.773457278], 'GSM1236647': [6.819829503, 6.737606381, 6.810705227666666, 6.804969095, 6.846374458], 'GSM1236648': [6.668914853, 6.859453332, 6.7353106086666665, 6.963283672, 6.644466189], 'GSM1236649': [6.715163609, 6.79712341, 6.739233725000001, 7.0191843, 6.68488119], 'GSM1236650': [6.725004828, 6.817735326, 6.710829740333334, 6.923969432, 6.690244447], 'GSM1236651': [6.719343433, 6.821149953, 6.778273417, 6.975174776, 6.843984104], 'GSM1236652': [6.813891586, 6.957789136, 6.761183182666667, 6.828386366, 6.815515253], 'GSM1236653': [6.753816743, 6.817994805, 6.735016162666667, 7.15422604, 6.656622738], 'GSM1236654': [6.734399654, 6.79112331, 6.727573143333333, 6.889305167, 6.712708704], 'GSM1236655': [6.801661273, 6.78896589, 6.7563429269999995, 6.937591846, 6.685064167], 'GSM1236656': [6.596197386, 6.788229024, 6.721482100999999, 6.92445524, 6.785945545], 'GSM1236657': [6.7648

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Type_1_Diabetes_Risk/gene_data/GSE51058.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Type_1_Diabetes_Risk')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE51058', './preprocessed/Type_1_Diabetes_Risk/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Type_1_Diabetes_Risk/GSE51058.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Type_1_Diabetes_Risk', the least common label is '0.0' with 15 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Type_1_Diabetes_Risk' in this dataset is fine.

